In [ ]:
# https://datanoon.com/blog/loading_data_rest_api_to_spark/

In [1]:
import requests
import json

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession

conf = SparkConf().setAppName("app1").setMaster("local")
sc = SparkContext('local', 'CurrentWeather')
spark = SparkSession(sc)

In [5]:
# function to make the call to the REST API service

def get_weather():
    url = f"https://api.openweathermap.org/data/2.5/weather?q=barcelona&appid=24b234cb23d2bddfc71ee6db0e1b1d6c"
    response = requests.get(url)
    return response

In [13]:
weather = get_weather()

# visualize the request results
print(weather.text)

{"coord":{"lon":2.16,"lat":41.39},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01n"}],"base":"stations","main":{"temp":279.57,"feels_like":277.09,"temp_min":277.15,"temp_max":280.93,"pressure":994,"humidity":70},"visibility":10000,"wind":{"speed":1,"deg":300},"clouds":{"all":0},"dt":1607114959,"sys":{"type":1,"id":6398,"country":"ES","sunrise":1607065291,"sunset":1607098936},"timezone":3600,"id":3128760,"name":"Barcelona","cod":200}


In [14]:
# creating a df from the json and saving the results
json_rdd = sc.parallelize([weather.text])
json_df = spark.read.json(json_rdd)
json_df.show()
json_df.rdd.saveAsTextFile('current_weather')

+--------+------+---+-------------+----------+-------+--------------------+---------+--------------------+--------+----------+--------------------+--------+
|    base|clouds|cod|        coord|        dt|     id|                main|     name|                 sys|timezone|visibility|             weather|    wind|
+--------+------+---+-------------+----------+-------+--------------------+---------+--------------------+--------+----------+--------------------+--------+
|stations|   [0]|200|[41.39, 2.16]|1607114959|3128760|[277.09, 70, 994,...|Barcelona|[ES, 6398, 160706...|    3600|     10000|[[clear sky, 01n,...|[300, 1]|
+--------+------+---+-------------+----------+-------+--------------------+---------+--------------------+--------+----------+--------------------+--------+



In [16]:
sc.stop() 